In [1]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, average_precision_score, accuracy_score
import csv

In [2]:
def get_algo_path():
    import os
    cwd = os.getcwd()
    s='\\'
    if '\\' not in cwd:
        s='/'
    cwd= cwd.split(s)[4:-1]
    cwd.append('Not Normalized')
    cwd='/'.join(cwd)
    return cwd

def get_csv_path():
    import os
    cwd = os.getcwd()
    s="\\"
    if "\\" not in cwd:
        s='/'
    file= cwd.split(s)[:4]
    file.append('models_scores.csv')
    file= s.join(file)
    return file

def line_is_exist(file, row):
    logfile = open(file, 'r')
    loglist = logfile.readlines()
    logfile.close()
    for line in loglist:
        if ','.join(row) in line:
            return True
    return False

def write_new_score(file, line):
    if( not line_is_exist(file, line) ):
        with open(file, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(line)
    else:
        print('line exsist already')
    

In [3]:
data= pd.read_csv('../data.csv')

In [4]:
##copying data
data1=data.copy()
### spliting data en X et Y
X= data1.drop('Loan Status', axis=1)
Y= data1['Loan Status']
### spliting the data to train and test
X_train , X_test , Y_train , Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 123)
## dicision tree (CART)
dct = DecisionTreeClassifier()
params={'criterion' : ['gini', 'entropy'],
        'max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14, 15, 16, 17, None]}

##accuracy
grid_search_acc= GridSearchCV(estimator=dct, param_grid= params, scoring='accuracy',cv=2, n_jobs=-1)
grid_search_acc= grid_search_acc.fit(X_train, Y_train)
y_predict= grid_search_acc.best_estimator_.predict(X_test)


In [5]:
## get avg precision & avg recall
report= classification_report(Y_test, y_predict, output_dict=True)
avg_list = report.pop("weighted avg")
avg_precision= round(avg_list['precision'], 3)
avg_recall= round(avg_list['recall'], 3)
accuraccy= round(accuracy_score(Y_test,y_predict), 3)
## csv row
csv_row = [get_algo_path(), 'Decision Tree', str(grid_search_acc.best_params_), str(accuraccy), str(avg_precision), str(avg_recall)]
## write file
csv_file= get_csv_path()
write_new_score(csv_file, csv_row)

In [6]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_acc.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")


----------
best parameters:  {'criterion': 'gini', 'max_depth': 12}
----------
[[14114  5835]
 [  911 19047]]
----------
              precision    recall  f1-score   support

           0       0.94      0.71      0.81     19949
           1       0.77      0.95      0.85     19958

    accuracy                           0.83     39907
   macro avg       0.85      0.83      0.83     39907
weighted avg       0.85      0.83      0.83     39907

----------


In [7]:
grid_search_acc.cv_results_



{'mean_fit_time': array([0.41738284, 0.46076739, 0.55800819, 0.64327908, 0.75797272,
        0.83077979, 0.9190433 , 0.93799233, 0.96591604, 1.0457027 ,
        1.01977479, 1.22023582, 1.17585552, 1.18183899, 1.28007674,
        0.57895255, 0.66322529, 0.72406209, 0.80983388, 0.85720778,
        0.94995964, 1.01079702, 1.06814528, 1.12798464, 1.24168026,
        1.27309513, 1.36684537, 1.3902818 , 1.31448472, 1.40374506]),
 'std_fit_time': array([3.48865986e-03, 3.98957729e-03, 2.84249783e-02, 6.38301373e-02,
        6.68210983e-02, 3.59051228e-02, 1.24664307e-02, 1.24666691e-02,
        2.14434862e-02, 1.28156424e-01, 1.74528360e-02, 7.62941837e-02,
        8.97562504e-03, 2.29392052e-02, 1.34634972e-02, 1.84504986e-02,
        2.99199820e-02, 1.19689703e-02, 1.99466944e-02, 1.84509754e-02,
        1.44611597e-02, 1.34634972e-02, 4.76837158e-07, 9.97316837e-03,
        1.99462175e-02, 1.04720592e-02, 1.64568424e-02, 3.88952494e-02,
        2.29402781e-02, 1.14690065e-02]),
 'mean_scor